# PROJECT NAME HERE (to be updated)

#### PROJECT SPECIFICATION
- **Data Set:** Student Performance Data Set
- **Question:** question here

#### INTRODUCTION
- Provide some relevant background information on the topic so that someone unfamiliar with it will be prepared to understand the rest of your proposal
- Clearly state the question you will try to answer with your project
- Identify and describe the dataset that will be used to answer the question

#### PRELIMINARY EXPLORATORY DATA ANALYSIS

In [20]:
# loads the library needed to read the data
library(tidyverse)

In [21]:
read_csv("data/student-mat.csv")

ERROR: Error: 'data/student-mat.csv' does not exist in current working directory ('').


#### METHODS

#### EXPEECTED OUTCOMES AND SIGNIFICANCE